<a href="https://colab.research.google.com/github/babo989/Pyrocystis-Tracking/blob/main/Efficientnet_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Import necessary libraries
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
from PIL import Image
from PIL import UnidentifiedImageError
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

"""
Get the path to the cached TensorFlow Hub model and delete the cached directory
"""

model_cache_path = hub.resolve("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1")
shutil.rmtree(model_cache_path)

"""
URL for the EfficientNet B0 model from TensorFlow Hub
"""

model_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

"""
Create a sequential model using the EfficientNet B0 model as the feature extractor
and a dense layer with 6 output units (corresponding to the number of classes) and softmax activation
"""

model = tf.keras.Sequential([
hub.KerasLayer(model_url, trainable=False),
tf.keras.layers.Dense(6, activation='softmax')
])

"""
Function to preprocess an image by resizing and normalizing pixel values
"""

def preprocess_image(image_path):
image = Image.open(image_path).convert("RGB")
image = image.resize((224, 224)) # EfficientNet input size
image = np.array(image) / 255.0
return image

"""
Dictionary mapping class names to numerical labels
"""

class_mapping = {'Dead': 0, 'Veg': 1, 'Div': 2, 'PreDiv': 3, 'Spore': 4, 'New': 5}

"""
Function to load image data and labels from a directory structure
"""

def load_data_from_folders(folder_path):
data = []
labels = []
class_names = sorted(os.listdir(folder_path))
for i, class_name in enumerate(class_names):
if class_name == '.DS_Store':
continue # Skip the .DS_Store file
class_folder = os.path.join(folder_path, class_name)
for image_file in os.listdir(class_folder):
image_path = os.path.join(class_folder, image_file)
try:
image = preprocess_image(image_path)
data.append(image)
labels.append(class_mapping[class_name]) # Assign a label corresponding to the class index
except (OSError, UnidentifiedImageError):
print(f"Skipping invalid image file: {image_path}")
return np.array(data), np.array(labels)

"""
Load data from the specified directory
"""

data, labels = load_data_from_folders("/Users/adam/Documents/Data/Planktoscope/Pyro_classifiertest/Classification")

"""
Split the data into training/validation and test sets
"""

X_train_val, X_test, y_train_val, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

"""
Further split the training/validation set into separate training and validation sets
"""

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

"""
Create an instance of ImageDataGenerator for data augmentation
"""

datagen = ImageDataGenerator(
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)

"""
Compute statistics for data augmentation based on the training data
"""

datagen.fit(X_train)

"""
Compile the model with the specified loss function, optimizer, and metrics
"""

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

"""
Set the batch size and number of epochs
"""

batch_size = 32
epochs = 10

"""
Train the model with data augmentation on the training set and evaluate on the validation set
"""

model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(X_val, y_val))

"""
Evaluate the trained model on the test set
"""

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

"""
Function to classify objects in an image using the trained model
"""

def classify_objects(image_path, model):
image = preprocess_image(image_path)
image = np.expand_dims(image, axis=0)
predictions = model.predict(image)
class_index = np.argmax(predictions)
return class_index, predictions

"""
Save the trained model to a file
"""

model.save("/Users/adam/Documents/Data/Planktoscope/Pyro_classifiertest/Classification4.h5")

"""
Print a summary of the model architecture
"""

model.summary()

"""
Print the unique class labels present in the dataset
"""

print(np.unique(labels))

"""
Example usage: classify an image
"""

image_path = "/Users/adam/Documents/Data/Planktoscope/Pyro_classifiertest/10_28_14_574045_5.jpg"
class_index, predictions = classify_objects(image_path, model)
print("Predicted class index:", class_index)
print("Class probabilities:", predictions)